In [1]:
import os

import numpy as np
import pandas as pd

import cloudpickle

from transformers import *
import torch

2021-11-26 20:15:24.408314: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-11-26 20:15:24.408434: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/opt/conda/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [2]:
matchings = pd.read_csv('../input/test-combiner-target-dataset/final_matchings.csv', keep_default_na=False)

print(matchings.shape)

matchings.head(3)

(92366, 22)


,page_title,caption_reference_description,page_title_translit,caption_translit,page_title_lang,page_title_lang_p,page_title_en,caption_lang,caption_lang_p,caption_en,...,page_title_contains_digit,undigit_page_title,target,target_id,caption_translation,final_caption,PREfinal_caption,page_title_translation,final_page_title,PREfinal_page_title
0,Albert Pike,Albert Pike,,,en,0.124504,True,en,0.124504,True,...,False,Albert Pike,Albert Pike [SEP] Albert Pike,0,,Albert Pike,Albert Pike,,Albert Pike,Albert Pike
1,Anna Blount,"Blount and her young daughter Ruth, in 1911",,,en,0.739186,True,en,0.971129,True,...,False,Anna Blount,Anna Blount [SEP] Blount and her young daughte...,1,,"Blount and her young daughter Ruth, in 1911","Blount and her young daughter Ruth, in 1911",,Anna Blount,Anna Blount
2,Río Marañón,"Die Río Marañón in die Huánuco-streek, Peru",Rio Maranon,"Die Rio Maranon in die Huanuco-streek, Peru",es,0.684134,False,de,0.376005,False,...,False,Río Marañón,Río Marañón [SEP] Die Río Marañón in die Huánu...,2,"Die Río Marañón in die Huánuco-streek, Peru","Die Río Marañón in die Huánuco-streek, Peru","Die Río Marañón in die Huánuco-streek, Peru",Marañón River,Marañón River,Marañón River


In [3]:
model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

label_list = [
    "O",       # Outside of a named entity
    "B-MISC",  # Beginning of a miscellaneous entity right after another miscellaneous entity
    "I-MISC",  # Miscellaneous entity
    "B-PER",   # Beginning of a person's name right after another person's name
    "I-PER",   # Person's name
    "B-ORG",   # Beginning of an organisation right after another organisation
    "I-ORG",   # Organisation
    "B-LOC",   # Beginning of a location right after another location
    "I-LOC"    # Location
]

Downloading:   0%|          | 0.00/998 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [4]:
def retrieve_entities(pairs, verbose=False):
    def repair(s):
        return s.replace(' #', '').replace('#', '')
    def unique(l):
        return list(set(l))
    
    
    reading_entity = False
    current_entity_type = None
    
    misc, per, org, loc = [], [], [], []
    
    entity_parts = []
    
    for part, et in pairs:
        if not reading_entity and et == 'O':
            continue
        else:
            if not reading_entity:
                current_et = et
                reading_entity = True               
                
            if current_et == et:
                entity_parts.append(part)
            else:
                repaired_entity = repair(' '.join(entity_parts))
                
                if verbose:
                    print(repaired_entity)
                    
                locals()[current_et.split('-')[-1].lower()].append(repaired_entity)
                entity_parts.clear()
                
                if et == 'O':
                    reading_entity = False
                    current_et = None
                else:
                    entity_parts.append(part)
                    current_et = 'I-' + et.split('-')[-1]
                    
                    
    return { 'misc': unique(misc), 'per': unique(per), 'org': unique(org), 'loc': unique(loc)}

In [5]:
image_entities = { }

for index, row in matchings.iterrows():
    try:
        sequence = row['final_page_title'].strip()
        
        if sequence != '':
            tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
            inputs = tokenizer.encode(sequence, return_tensors="pt")

            outputs = model(inputs)[0]
            predictions = torch.argmax(outputs, dim=2)

            l = [(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].tolist())]

            image_entities[index] = retrieve_entities(l)
    except Exception as ex:
        print(index, sequence)
        print(str(ex))
        print()

In [6]:
with open('final_page_title_entities.dump', 'wb') as file:
    cloudpickle.dump(image_entities, file)